In [11]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.style.use('classic')

In [12]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

In [13]:
import os
import cv2
from PIL import Image
import numpy as np

In [14]:
image_directory = '299aug-nor-macular/'
SIZE = 150
dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.  
label = []  #Place holders to define add labels. We will add 0 to all parasitized images and 1 to uninfected.

In [15]:
macular_degeneration_images = os.listdir(image_directory + 'macular-degeneration/')
for i, image_name in enumerate(macular_degeneration_images):    #Remember enumerate method adds a counter and returns the enumerate object
    
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'macular-degeneration/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)

#Iterate through all images in Uninfected folder, resize to 64 x 64
#Then save into the same numpy array 'dataset' but with label 1

In [16]:
normal_images = os.listdir(image_directory + 'normal/')
for i, image_name in enumerate(normal_images):
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'normal/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)

dataset = np.array(dataset)
label = np.array(label)

In [17]:
from sklearn.model_selection import train_test_split
#from keras.utils import to_categorical

In [18]:
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.20, random_state = 0)

In [19]:
#Without scaling (normalize) the training may not converge. 
#Normalization is a rescaling of the data from the original range 
#so that all values are within the range of 0 and 1.
from tensorflow.keras.utils import normalize
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

#Do not do one-hot encoding as it generates a shape of (num, 2)
#But the network expects an input of (num, 1) for the last layer for binary classification
#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

In [20]:
###2 conv and pool layers. with some normalization and drops in between.

INPUT_SHAPE = (SIZE, SIZE, 3)   #change to (SIZE, SIZE, 3)


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(1))
model.add(Activation('sigmoid'))  
#Do not use softmax for binary classification
#Softmax is useful for mutually exclusive classes, either cat or dog but not both.
#Also, softmax outputs all add to 1. So good for multi class problems where each
#class is given a probability and all add to 1. Highest one wins. 

In [21]:
#Sigmoid outputs probability. Can be used for non-mutually exclusive problems.
#But, also good for binary mutually exclusive (cat or not cat). 

model.compile(loss='binary_crossentropy',
              optimizer='adam',             #also try adam
              metrics=['accuracy'])


In [22]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        1

In [23]:
history = model.fit(X_train, 
                         y_train, 
                         batch_size = 64, 
                         verbose = 1, 
                         epochs = 10,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/10
100/100 [==============================] - 49s 490ms/step - loss: 0.5061 - accuracy: 0.7292 - val_loss: 0.3855 - val_accuracy: 0.8250
Epoch 2/10
100/100 [==============================] - 49s 495ms/step - loss: 0.3798 - accuracy: 0.8295 - val_loss: 0.3609 - val_accuracy: 0.8406
Epoch 3/10
100/100 [==============================] - 50s 504ms/step - loss: 0.3478 - accuracy: 0.8511 - val_loss: 0.3273 - val_accuracy: 0.8569
Epoch 4/10
100/100 [==============================] - 51s 506ms/step - loss: 0.3188 - accuracy: 0.8655 - val_loss: 0.3107 - val_accuracy: 0.8650
Epoch 5/10
100/100 [==============================] - 51s 515ms/step - loss: 0.2947 - accuracy: 0.8745 - val_loss: 0.3220 - val_accuracy: 0.8619
Epoch 6/10
100/100 [==============================] - 52s 522ms/step - loss: 0.2905 - accuracy: 0.8798 - val_loss: 0.3115 - val_accuracy: 0.8656
Epoch 7/10
100/100 [==============================] - 51s 508ms/step - loss: 0.2850 - accuracy: 0.8813 - val_loss: 0.3033 - val_ac

In [25]:
history = history.model.fit(X_train, 
                         y_train, 
                         batch_size = 64, 
                         verbose = 1, 
                         epochs = 10,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/10
100/100 [==============================] - 59s 590ms/step - loss: 0.2341 - accuracy: 0.9020 - val_loss: 0.2696 - val_accuracy: 0.8838
Epoch 2/10
100/100 [==============================] - 61s 610ms/step - loss: 0.2221 - accuracy: 0.9050 - val_loss: 0.2684 - val_accuracy: 0.8775
Epoch 3/10
100/100 [==============================] - 60s 603ms/step - loss: 0.2182 - accuracy: 0.9081 - val_loss: 0.3296 - val_accuracy: 0.8481
Epoch 4/10
100/100 [==============================] - 58s 581ms/step - loss: 0.2166 - accuracy: 0.9083 - val_loss: 0.2804 - val_accuracy: 0.8725
Epoch 5/10
100/100 [==============================] - 55s 550ms/step - loss: 0.2008 - accuracy: 0.9137 - val_loss: 0.2765 - val_accuracy: 0.8806
Epoch 6/10
100/100 [==============================] - 58s 576ms/step - loss: 0.1844 - accuracy: 0.9245 - val_loss: 0.2861 - val_accuracy: 0.8819
Epoch 7/10
100/100 [==============================] - 54s 536ms/step - loss: 0.1679 - accuracy: 0.9330 - val_loss: 0.2820 - val_ac

In [26]:
history = history.model.fit(X_train, 
                         y_train, 
                         batch_size = 64, 
                         verbose = 1, 
                         epochs = 10,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/10
100/100 [==============================] - 53s 526ms/step - loss: 0.1287 - accuracy: 0.9469 - val_loss: 0.3626 - val_accuracy: 0.8669
Epoch 2/10
100/100 [==============================] - 59s 591ms/step - loss: 0.1265 - accuracy: 0.9495 - val_loss: 0.3228 - val_accuracy: 0.8831
Epoch 3/10
100/100 [==============================] - 62s 624ms/step - loss: 0.1118 - accuracy: 0.9567 - val_loss: 0.3824 - val_accuracy: 0.8744
Epoch 4/10
100/100 [==============================] - 54s 541ms/step - loss: 0.1035 - accuracy: 0.9631 - val_loss: 0.3895 - val_accuracy: 0.8781
Epoch 5/10
100/100 [==============================] - 59s 588ms/step - loss: 0.0920 - accuracy: 0.9630 - val_loss: 0.4663 - val_accuracy: 0.8675
Epoch 6/10
100/100 [==============================] - 56s 559ms/step - loss: 0.0816 - accuracy: 0.9688 - val_loss: 0.4063 - val_accuracy: 0.8781
Epoch 7/10
100/100 [==============================] - 58s 580ms/step - loss: 0.0788 - accuracy: 0.9706 - val_loss: 0.4235 - val_ac